<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Mortgage Default Machine Learning Model</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
</table>

This notebook walks you through these steps:
- Access the data
- Cleanse data for analysis
- Explore data
- Build a classification model
- Save the model in the ML repository with associated meta data


### Step 1: Load Data as Spark Dataframe

Read the three Mortgage files - Insert them as SPARK Dataframes

In [24]:
import os
from pyspark.sql import SQLContext
# Add asset from file system
df_data_1 = SQLContext(sc).read.csv(os.environ['DSX_PROJECT_DIR']+'/datasets/Mortgage_Customer.csv', header='true', inferSchema = 'true')

In [25]:
# Add asset from file system
df_data_2 = SQLContext(sc).read.csv(os.environ['DSX_PROJECT_DIR']+'/datasets/Mortgage_Property.csv', header='true', inferSchema = 'true')

In [26]:
# Add asset from file system
df_data_3 = SQLContext(sc).read.csv(os.environ['DSX_PROJECT_DIR']+'/datasets/Mortgage_Default.csv', header='true', inferSchema = 'true')

In [27]:
customer = df_data_1
property = df_data_2
default = df_data_3

In [28]:
customer.cache()
property.cache()
default.cache()

DataFrame[ID: int, MortgageDefault: string]

In [29]:
print("Customer dataframe contains these fields:")
print(customer.schema.names)
print("")
print("Property dataframe contains these fields:")
print(property.schema.names)
print("")
print("Default dataframe contains these fields:")
print(default.schema.names)

Customer dataframe contains these fields:
['ID', 'Income', 'AppliedOnline', 'Residence', 'Yrs_at_Current_Address', 'Yrs_with_Current_Employer', 'Number_of_Cards', 'Creditcard_Debt', 'Loans', 'Loan_Amount']

Property dataframe contains these fields:
['ID', 'SalePrice', 'Location']

Default dataframe contains these fields:
['ID', 'MortgageDefault']


View the data within the three data frames created above. 

In [30]:
#customer.show(5)
#property.show(5)
default.show(5)

+------+---------------+
|    ID|MortgageDefault|
+------+---------------+
|101600|            YES|
|101731|             NO|
|100548|            YES|
|101472|             NO|
|100562|            YES|
+------+---------------+
only showing top 5 rows



### Step 2: Merge Files

In [31]:
# Join the Customer, Property and Default tables together with ID being the key field.
# Keep all fields from the Customer and Default tables but only SalePrice and Location from the Property table

merged = customer.join(property, customer['ID'] == property['ID'])\
                   .join(default, customer['ID']==default['ID'])\
                   .select(customer['ID'],customer['Income'],customer['AppliedOnline'],customer['Residence'],\
                           customer['Yrs_at_Current_Address'],customer['Yrs_with_Current_Employer'],customer['Number_of_Cards'],\
                           customer['Creditcard_Debt'],customer['Loans'],customer['Loan_Amount'],\
                           property['SalePrice'], property['Location'], default['MortgageDefault'])

# Preview  5 rows
merged.toPandas().head()

,ID,Income,AppliedOnline,Residence,Yrs_at_Current_Address,Yrs_with_Current_Employer,Number_of_Cards,Creditcard_Debt,Loans,Loan_Amount,SalePrice,Location,MortgageDefault
0,100522,43982,YES,Owner Occupier,13,11,2,1055,0,9405,500000,110,NO
1,101756,59944,YES,Owner Occupier,20,11,2,3894,0,9880,750000,110,NO
2,101354,57718,YES,Owner Occupier,25,16,2,1555,1,6285,155000,130,YES
3,100512,45621,YES,Owner Occupier,1,19,1,1878,0,9260,195000,100,YES
4,100537,45081,NO,Owner Occupier,14,15,2,713,1,8430,140000,110,NO


### Step 3: Simple Data Preparation - Rename some columns and ensure correct data types 
This step is to remove spaces from columns names

In [32]:
merged = merged.withColumnRenamed("Yrs_at_Current_Address", "YearCurrentAddress").withColumnRenamed("Yrs_with_Current_Employer","YearsCurrentEmployer")\
                .withColumnRenamed("Number_of_Cards","NumberOfCards").withColumnRenamed("Creditcard_Debt","CCDebt").withColumnRenamed("Loan_Amount", "LoanAmount")
merged.toPandas().head(3)

,ID,Income,AppliedOnline,Residence,YearCurrentAddress,YearsCurrentEmployer,NumberOfCards,CCDebt,Loans,LoanAmount,SalePrice,Location,MortgageDefault
0,100522,43982,YES,Owner Occupier,13,11,2,1055,0,9405,500000,110,NO
1,101756,59944,YES,Owner Occupier,20,11,2,3894,0,9880,750000,110,NO
2,101354,57718,YES,Owner Occupier,25,16,2,1555,1,6285,155000,130,YES


Check data types and re-cast numeric fields to **Integers**

In [33]:
merged.dtypes

[('ID', 'int'),
 ('Income', 'int'),
 ('AppliedOnline', 'string'),
 ('Residence', 'string'),
 ('YearCurrentAddress', 'int'),
 ('YearsCurrentEmployer', 'int'),
 ('NumberOfCards', 'int'),
 ('CCDebt', 'int'),
 ('Loans', 'int'),
 ('LoanAmount', 'int'),
 ('SalePrice', 'int'),
 ('Location', 'int'),
 ('MortgageDefault', 'string')]

In [34]:
merged = merged.select(merged.ID.cast("integer"),merged.Income.cast('integer'),merged.AppliedOnline,merged.Residence,\
                   merged.YearCurrentAddress.cast('integer'),merged.YearsCurrentEmployer.cast('integer'),\
                   merged.NumberOfCards.cast('integer'),merged.CCDebt.cast('integer'),merged.Loans.cast('integer'),\
                   merged.LoanAmount.cast('integer'),merged.SalePrice,merged.Location,merged.MortgageDefault)
merged.dtypes

[('ID', 'int'),
 ('Income', 'int'),
 ('AppliedOnline', 'string'),
 ('Residence', 'string'),
 ('YearCurrentAddress', 'int'),
 ('YearsCurrentEmployer', 'int'),
 ('NumberOfCards', 'int'),
 ('CCDebt', 'int'),
 ('Loans', 'int'),
 ('LoanAmount', 'int'),
 ('SalePrice', 'int'),
 ('Location', 'int'),
 ('MortgageDefault', 'string')]

### Step 4: Data Exploration

1) Obtain some data shape summaries in terms of number of fields and records <br>
2) Perform some exploratory analysis of distributions, scatterplots using two different graphics packages

#### Feel free to play around with the charts to explore other features that are in the data set

In [35]:
print("There are " + str(merged.count()) + " records and " + str(len(merged.columns)) + " fields in the dataset.")

There are 419 records and 13 fields in the dataset.


In [36]:
import brunel
df = merged.toPandas()
%brunel data('df') bar x(Residence) y(Income) mean(CCDebt) color(MortgageDefault) stack tooltip(Income) | x(YearCurrentAddress) y(YearsCurrentEmployer) point color(MortgageDefault) tooltip(YearCurrentAddress, YearsCurrentEmployer) :: width=1100, height=400 

<IPython.core.display.Javascript object>

In [37]:
from pixiedust.display import *
display(merged)

### Step 5: Build the Spark pipeline and the Random Forest model
"Pipeline" is an API in SparkML that's used for building models.
Additional information on SparkML: https://spark.apache.org/docs/2.0.2/ml-guide.html

In [38]:
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# Prepare string variables so that they can be used by the algorithm
stringIndexer1 = StringIndexer(inputCol='AppliedOnline', outputCol='AppliedOnlineEncoded')
stringIndexer2 = StringIndexer(inputCol='Residence',outputCol='ResidenceEncoded')
stringIndexer3 = StringIndexer(inputCol='MortgageDefault', outputCol='label')

# Instanciate the algorithm
rf=RandomForestClassifier(labelCol="label", featuresCol="features")


# Pipelines API requires that input variables are passed in  a vector
assembler = VectorAssembler(inputCols=["Income", "AppliedOnlineEncoded", "ResidenceEncoded", "YearCurrentAddress", "YearsCurrentEmployer", "NumberOfCards", \
                                       "CCDebt", "Loans", "LoanAmount", "SalePrice", "Location"], outputCol="features")

pipeline = Pipeline(stages=[stringIndexer1, stringIndexer2, stringIndexer3, assembler, rf])

In [39]:
# Split data into train and test datasets
train, test = merged.randomSplit([80.0,20.0], seed=6)

In [40]:
# Build model based upon the pipeline defined in the above cell
model = pipeline.fit(train)

### Step 6: Score the test data set

In [41]:
results = model.transform(test)
results.toPandas().head(3)

,ID,Income,AppliedOnline,Residence,YearCurrentAddress,YearsCurrentEmployer,NumberOfCards,CCDebt,Loans,LoanAmount,SalePrice,Location,MortgageDefault,AppliedOnlineEncoded,ResidenceEncoded,label,features,rawPrediction,probability,prediction
0,100282,45715,YES,Owner Occupier,8,14,2,772,1,12985,137000,100,NO,0.0,0.0,0.0,"[45715.0, 0.0, 0.0, 8.0, 14.0, 2.0, 772.0, 1.0...","[16.035739057141832, 3.964260942858168]","[0.8017869528570916, 0.19821304714290838]",0.0
1,100284,45049,YES,Public Housing,6,16,2,1345,1,9085,280000,110,NO,0.0,2.0,0.0,"[45049.0, 0.0, 2.0, 6.0, 16.0, 2.0, 1345.0, 1....","[9.196965147856195, 10.803034852143803]","[0.45984825739280977, 0.5401517426071901]",1.0
2,100285,44974,YES,Public Housing,14,18,2,2772,0,9515,264000,130,NO,0.0,2.0,0.0,"[44974.0, 0.0, 2.0, 14.0, 18.0, 2.0, 2772.0, 0...","[8.908459463913955, 11.091540536086047]","[0.44542297319569774, 0.5545770268043023]",1.0


### Step 7: Model Evaluation 

In [42]:
print('Precision model1 = {:.2f}.'.format(results.filter(results.label == results.prediction).count() / float(results.count())))

Precision model1 = 0.62.


In [43]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")
print('Area under ROC curve = {:.2f}.'.format(evaluator.evaluate(results)))

Area under ROC curve = 0.62.


In [44]:
evaluatorRF = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
accuracy = evaluatorRF.evaluate(results)

print("Accuracy = %g" % accuracy)

Accuracy = 0.615079


### Step 8: Save Model in ML repository


In [45]:
import ssl
ssl.match_hostname = lambda cert, hostname: True

from dsx_ml.ml import save

import datetime

now = datetime.datetime.now()

model_name = "Predict Mortgage Default LOS "+now.strftime("%Y-%m-%d %H%M%S")
save(name = model_name,
     model = model,
     algorithm_type = 'Classification',
     test_data = test)

{'path': '/user-home/999/DSX_Projects/mortgage-analysis-new/models/Predict Mortgage Default LOS/1',
 'scoring_endpoint': 'https://dsxl-api/v3/project/score/Python36/spark-2.3/mortgage-analysis-new/Predict%20Mortgage%20Default%20LOS/1'}

## Summary:
You are now at the end of this notebook and should have successfully:
- Performed basic data preparation on the loaded data
- Explored the data graphically
- Build a Spark Model in the form of a pipeline
- Evaluated the model for accuracy levels
- Stored the model into the IBM ML environment, making it ready for deployment